In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/System-Threat-Forecaster/sample_submission.csv
/kaggle/input/System-Threat-Forecaster/train.csv
/kaggle/input/System-Threat-Forecaster/test.csv


# Importing Libraries

In [3]:
from sklearn.model_selection import train_test_split,RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, OrdinalEncoder,FunctionTransformer, MaxAbsScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC,LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectKBest, f_regression, f_classif
from sklearn.impute import KNNImputer
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import VarianceThreshold

from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import roc_curve, auc

## Data Loading

In [7]:
#Loading the csv file using pandas in the req. variables.
df_train = pd.read_csv("/kaggle/input/System-Threat-Forecaster/train.csv")

df_test = pd.read_csv("/kaggle/input/System-Threat-Forecaster/test.csv")

## Exploratory Data Analysis

In [ ]:
# import pandas as pd
# import scipy.stats as ss

# # Function to compute Chi-Squared test results
# def chi_squared_test(df):
#     cat_cols = df.select_dtypes(include=['object', 'category']).columns
#     chi2_matrix = pd.DataFrame(index=cat_cols, columns=cat_cols)

#     for col1 in cat_cols:
#         for col2 in cat_cols:
#             if col1 == col2:
#                 chi2_matrix.loc[col1, col2] = 1.0  # Identity correlation
#             else:
#                 contingency_table = pd.crosstab(df[col1], df[col2])
#                 chi2, p, _, _ = ss.chi2_contingency(contingency_table)
#                 chi2_matrix.loc[col1, col2] = p  # P-value

#     return chi2_matrix

# # Perform Chi-Squared test
# chi2_results = chi_squared_test(df_train)
# print("Chi-Squared Test P-values:\n", chi2_results)

In [ ]:
# import pandas as pd
# import numpy as np
# import scipy.stats as stats
# from sklearn.preprocessing import LabelEncoder

# # Function to compute Cramér’s V
# def cramers_v(x, y):
#     confusion_matrix = pd.crosstab(x, y)
#     chi2 = stats.chi2_contingency(confusion_matrix)[0]
#     n = confusion_matrix.sum().sum()
#     phi2 = chi2 / n
#     r, k = confusion_matrix.shape
#     phi2corr = max(0, phi2 - ((k-1)*(r-1)) / (n-1))  # Bias correction
#     rcorr = r - ((r-1)**2) / (n-1)
#     kcorr = k - ((k-1)**2) / (n-1)
#     return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

# # Load your dataset (change this to your actual dataset)
# df = df_test

# # Select only categorical features
# cat_features = df.select_dtypes(include=['object']).columns.tolist()

# # Encode categorical variables into numeric labels
# encoders = {col: LabelEncoder().fit(df[col].astype(str)) for col in cat_features}
# df_encoded = df.copy()
# for col in cat_features:
#     df_encoded[col] = encoders[col].transform(df[col].astype(str))

# # Compute Cramér’s V correlation matrix
# correlation_matrix = pd.DataFrame(index=cat_features, columns=cat_features)

# for col1 in cat_features:
#     for col2 in cat_features:
#         if col1 == col2:
#             correlation_matrix.loc[col1, col2] = 1.0  # Perfect correlation with itself
#         else:
#             correlation_matrix.loc[col1, col2] = cramers_v(df_encoded[col1], df_encoded[col2])

# # Convert values to numeric type
# correlation_matrix = correlation_matrix.astype(float)

# # Filter out features with low correlation with the target variable (e.g., threshold < 0.1)
# target = "YourTargetColumn"  # Change to actual target column
# if target in df.columns:
#     target_corr = correlation_matrix[target].drop(target, errors="ignore")
#     low_corr_features = target_corr[target_corr < 0.1].index.tolist()
#     print(f"Features with low correlation to {target}: {low_corr_features}")

# # Identify redundant features (highly correlated with other features, threshold > 0.8)
# redundant_features = set()
# threshold = 0.8
# for col1 in cat_features:
#     for col2 in cat_features:
#         if col1 != col2 and correlation_matrix.loc[col1, col2] > threshold:
#             redundant_features.add(col2)

# print(f"Highly correlated (redundant) features: {redundant_features}")

In [ ]:
#To find the Dataset shape.
print("The shape of the train data is",df_train.shape)
print("The shape of the test data is",df_test.shape)

In [ ]:
#To veiw the columns.
df_train.columns

In [ ]:
pd.set_option("display.max_colwidth", None)  
pd.set_option("display.max_rows", None)
df_train.nunique()

In [ ]:
#To view first few data of the train datas
df_train.head()

In [ ]:
#To find what types of datatypes are provided in the test data.
df_train.info()

In [ ]:
#'PrimaryDiskCapacityMB','SystemVolumeCapacityMB','TotalPhysicalRAMMB','InternalBatteryNumberOfCharges','PrimaryDisplayResolutionHorizontal','PrimaryDisplayResolutionVertical'

In [ ]:
df_train['MDC2FormFactor'].value_counts()

In [ ]:
#To find is there any null value or not.
df_train.isna().mean()*100

Here the data set has less np.nan values, so we can drop those coloumns if required or we can impute those columns using simpleImputer with strategy=mean and most_frequent

In [ ]:
#To veiw the statistics of the dataset.
df_train.describe()

From this table we can see the statistics of the DataFrame.

In [ ]:
#To find what kind of dataset problem we are working in and the Dataset is balanced or not.
df_train['target'].value_counts()

In [ ]:
sns.countplot(x=df_train['target'],data=df_train)

From the plot and the value counts of the target feature we can conclude that we are working in an binary classification problem. The dataset is almost balanced. 

In [ ]:
sns.countplot(x=df_train['target'],hue='MDC2FormFactor',data=df_train)

From the above plot we can conclue that the OSVersion does not play any vital role in the target column as they are almost equally distributed. So we can also drop this column

In [ ]:
df = df_train.select_dtypes(include=['number'])
print(df.var().sort_values())

The features having variance zero mentions that there is no change in the values so we can drop those features. Here we can drop features like 'AutoSampleSubmissionEnabled','IsFlightsDisabled','IsBetaUser'

In [ ]:
# import seaborn as sns
# df = df_train.select_dtypes(include=['number'])
# print(sns.heatmap(df.corr()))

In [ ]:
df_num = df_train.select_dtypes(include=['number'])
df_num.shape

In [ ]:
#df_train['target'][df_train['AutoSampleSubmissionEnabled']==4].value_counts()

In [ ]:
#df_num['IsBetaUser'].value_counts()

In [ ]:
# num_attribs=df_test.select_dtypes(include=['number']).columns
# for att in num_attribs:
#     plt.boxplot(df_train[att])
#     plt.title(att)
#     plt.show()

## 

# Data Cleaning/ Preprocessing

# Feature Engineering

In [ ]:
# df_train['totalpixel']=df_train['PrimaryDisplayResolutionVertical']*df_train['PrimaryDisplayResolutionHorizontal']
# df_test['totalpixel']=df_test['PrimaryDisplayResolutionVertical']*df_test['PrimaryDisplayResolutionHorizontal']

In [ ]:
# #converting date into integer format for train and test data.
# df_test['DateAS']=pd.to_datetime(df_test['DateAS'])
# df_train['DateAS']=pd.to_datetime(df_train['DateAS'])
# df_test['DateOS']=pd.to_datetime(df_test['DateOS'])
# df_train['DateOS']=pd.to_datetime(df_train['DateOS'])

# # Identify datetime columns
# datetime_cols = df_train.select_dtypes(include=['datetime64']).columns.tolist()

# # Convert datetime columns into numerical features
# def process_datetime(df, datetime_cols):
#     for col in datetime_cols:
#         #df[col] = pd.to_datetime(df[col], errors='coerce')  # Convert to datetime format
#         df[col + '_year'] = (df[col].dt.year)
#         df[col + '_month'] = (df[col].dt.month)
#         df[col + '_day'] = (df[col].dt.day)
#         df[col + '_weekday'] = (df[col].dt.weekday)
#         df[col + '_days_since_2000'] = (df[col] - pd.Timestamp("2000-01-01")).dt.days
# process_datetime(df_train,datetime_cols)
# process_datetime(df_test,datetime_cols)

In [ ]:
#converting date into integer format for train and test data.
df_test['DateAS']=pd.to_datetime(df_test['DateAS']).astype(int)
df_train['DateAS']=pd.to_datetime(df_train['DateAS']).astype(int)
df_test['DateOS']=pd.to_datetime(df_test['DateOS']).astype(int)
df_train['DateOS']=pd.to_datetime(df_train['DateOS']).astype(int)

In [ ]:
def cap_outliers(X, factor=1.5):
    X_capped = X.copy()
    for col_idx in range(X.shape[1]): 
        col = X[:, col_idx]
        Q1 = np.quantile(col, 0.25)
        Q3 = np.quantile(col, 0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - factor * IQR
        upper_bound = Q3 + factor * IQR
        X_capped[:, col_idx] = np.clip(col, lower_bound, upper_bound)
    return X_capped

In [8]:
#Dropping low variance numerical columns
df_test=df_test.drop(['AutoSampleSubmissionEnabled','IsFlightsDisabled','IsBetaUser','MachineID','ProductName','NumericOSVersion','DeviceFamily','FirmwareVersionID'],axis=1)
df_train=df_train.drop(['AutoSampleSubmissionEnabled','IsFlightsDisabled','IsBetaUser','MachineID','ProductName','NumericOSVersion','DeviceFamily','FirmwareVersionID'],axis=1)

In [ ]:
# columns_to_drop = [
#     "MachineID",  # Unique identifier (not useful for prediction)
#     "OEMNameID", "OEMModelID",  # Too many unique values (high cardinality)
#     "OSBuildLab",  # Redundant with OSBuildNumber
#     "OSBuildNumberOnly", "OSBuildRevisionOnly",  # Can be combined into OSBuildNumber
#     "GeoRegionID", "CityID",  # High cardinality, use CountryID instead
#     "LocaleEnglishNameID",  # Similar to OSUILocaleID (redundant)
#     "ProcessorManufacturerID", "ProcessorModelID",  # Covered by Processor and ProcessorCoreCount
#     "PrimaryDiskType",  # Often constant, low variance
#     "IsBetaUser",  # Low impact, rarely useful
#     "InternalBatteryNumberOfCharges",  # Often missing and not useful
#     "FlightRing",  # May not significantly impact malware detection
#     "FirmwareManufacturerID", "FirmwareVersionID",  # Not directly related to antivirus effectiveness
#     "IsPortableOS",  # Rarely relevant unless working on specialized OS versions
#     "OSGenuineState",  # May be redundant if LicenseActivationChannel exists
#     "SMode",  # Only relevant for certain Windows versions, check variance before dropping
#     "EnableLUA",  # If most values are the same, drop it
#     "ChassisType",  # May not be relevant unless correlated with malware risk
#     "IsGamer"  # May not have a strong correlation with security threats
# ]
# df_test.drop(columns=columns_to_drop,inplace=True,axis=1)
# df_train.drop(columns=columns_to_drop,inplace=True,axis=1)

In [9]:
num_attribs = df_test.select_dtypes(include=['number']).columns
cat_attribs = df_test.select_dtypes(include=['object']).columns

num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    # ('outlier_capper', FunctionTransformer(
    #     cap_outliers, 
    #     kw_args={'factor': 1.5}
    # )),  
    #('knn',KNNImputer(missing_values=np.nan,n_neighbors=5)),
    ('scaler', StandardScaler()),
])

cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    #('onehot', OneHotEncoder(handle_unknown='ignore')),
    ('ordinal',OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)),
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_attribs),
        ('cat', cat_transformer, cat_attribs),
    ]
)

In [10]:
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
])
model_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['RealTimeProtectionState', 'IsPassiveModeEnabled', 'AntivirusConfigID',
       'NumAntivirusProductsInstalled', 'NumAntivirusProductsEnabled',
       'HasTpm', 'CountryID', 'CityID', 'GeoRegionID'...
                                                  Index(['EngineVersion', 'AppVersion', 'SignatureVersion', 'PlatformType',
       'Processor', 'OSVersion', 'OsPlatformSubRelease', 'OSBuildLab',
       'SKUEditionName', 'MDC2FormFactor', 'PrimaryDiskType', 'ChassisType',
       'PowerPlatformRole', 'OSArchitecture', 'OSBranch', 'OSEdition',
       'OSSkuFriendlyName', 'OSInstallType', 'AutoUpdateOptionsName',
       'OSGenuineState', 'LicenseActivationChannel', 'FlightRing', 'DateAS',
       'DateOS'],
      dtype='object'))]))])

In [11]:
x = df_train.drop(['target'],axis=1)
y = df_train['target']

# Train Test Split

In [12]:
X_train, X_val, Y_train, Y_val = train_test_split(x, y, test_size=0.2, random_state=42)

In [13]:
model_pipeline.fit(X_train)
X_train_trans = model_pipeline.transform(X_train)
X_val_trans = model_pipeline.transform(X_val)
x_trans=model_pipeline.fit_transform(x)

## Models

# Logistic Regression

In [ ]:
# model = LogisticRegression(solver='liblinear', max_iter=1000)
# model.fit(X_train_trans,Y_train)
# Y_predicted = model.predict(X_val_trans)
# accuracy = accuracy_score(Y_predicted,Y_val)
# print(accuracy)

# Gradient Boosting

In [ ]:
# model = GradientBoostingClassifier(random_state=42, learning_rate=0.2, n_estimators=200,  max_depth=3, min_samples_split=2, min_samples_leaf=1)
# model.fit(X_train_trans,Y_train)
# Y_predicted = model.predict(X_val_trans)
# accuracy = accuracy_score(Y_predicted,Y_val)
# print(accuracy)

In [ ]:
# from sklearn.metrics import confusion_matrix
# matix=confusion_matrix(Y_predicted,Y_val)
# matix

# Random Forest Classifier

In [ ]:
# model = RandomForestClassifier()
# model.fit(X_train_trans,Y_train)
# Y_predicted = model.predict(X_val_trans)
# accuracy = accuracy_score(Y_predicted,Y_val)
# print(accuracy)

# XGBOOST

In [ ]:
model = XGBClassifier(
    use_label_encoder=False, 
    eval_metric='logloss',
    max_depth=8, 
    learning_rate=0.024553209024826435, 
    n_estimators=800, 
    subsample=0.9380090018464597, 
    colsample_bytree=0.5241721859195769, 
    min_child_weight=4, 
    gamma=0.20876688264239152,
    reg_alpha=1.2040716091162763e-07, 
    reg_lambda=8.917321918710645e-07,
    scale_pos_weight=0.8410588311336038,
    max_delta_step=5
)
# model.fit(X_train_trans, Y_train)
# Y_predicted = model.predict(X_val_trans)
# accuracy = accuracy_score(Y_predicted,Y_val)
# print(accuracy)
model.fit(x_trans,y)
model.score(x_trans,y)
#0.74876

After cleaning the data and preprocessing the data I had tried three models with that data namely Random Forest Classifier works based on the concept of bagging, Logistic Regression works as a baseline model which uses sigmoid function to classify graphs with 0.5 as threshold and lastly I had used one of the powerful technique called Gradient Boosting Classifier which works based on boosting technique form those three models Gradient boosting performs well.

In [ ]:
# from sklearn.model_selection import KFold, cross_val_score
# import numpy as np

# kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold CV

# scores = cross_val_score(model, X_train_trans, Y_train, cv=kf, scoring='accuracy')
# print("Cross-validation scores:", scores)
# print("Mean accuracy:", np.mean(scores))

In [ ]:
# model=MLPClassifier()
# model.fit(X_train_trans, Y_train)
# Y_predicted = model.predict(X_val_trans)
# accuracy = accuracy_score(Y_predicted,Y_val)
# print(accuracy)

## Submission

In [ ]:
test_data_trans = model_pipeline.transform(df_test)
test_predictions = model.predict(test_data_trans)
submission = pd.DataFrame({"id":range(0,df_test.shape[0]),
                          "target":test_predictions})
submission.to_csv("submission.csv",index=False)
print(submission.shape)

# Hyper Parameter Tuning 

In [ ]:
# # Define hyperparameter grid for GridSearchCV
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'learning_rate': [0.01, 0.1, 0.2],
#     'max_depth': [3, 5, 7],
#     'min_child_weight': [1, 3, 5],
#     'subsample': [0.7, 0.8, 0.9],
#     'colsample_bytree': [0.7, 0.8, 0.9]
# }

# # Perform Grid Search
# grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
# grid_search.fit(X_train_trans, Y_train)

# print("Best parameters from Grid Search:", grid_search.best_params_)

In [ ]:
# # Define hyperparameter distribution for RandomizedSearchCV
# param_dist = {
#     'n_estimators': [100, 200, 300, 400, 500],
#     'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
#     'max_depth': [3, 5, 7, 9],
#     'min_child_weight': [1, 3, 5, 7],
#     'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
#     'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0]
# }

# # Perform Randomized Search
# random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, 
#                                    scoring='accuracy', n_iter=20, cv=5, verbose=1, n_jobs=-1, random_state=42)
# random_search.fit(X_train_trans, Y_train)

# print("Best parameters from Randomized Search:", random_search.best_params_)

In [ ]:
# y_true = Y_val
# y_scores =  model.predict_proba(X_val_trans)[:, 1]

# # Calculate ROC curve
# fpr, tpr, thresholds = roc_curve(y_true, y_scores)
# roc_auc = auc(fpr, tpr)

# # Plot the ROC curve
# plt.figure()
# plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.legend(loc="lower right")
# plt.show()